# Exercise Sheet No. 7

---

> Machine Learning for Natural Sciences, Summer 2023, Jun.-Prof. Pascal Friederich
> 
> Deadline: 12.06.2023, 8am
>
> Container version: 1.0.2
>
> Tutor: navid.haghmoradi@kit.edu
---
**Topic**: This exercise sheet will use neural networks for a molecular dynamics simulation

# Molecular dynamics simulation

"Molecular dynamics (MD) is a computer simulation method for analyzing the physical movements of atoms and molecules [that are interacting to each other and creating forces among themselves]. (...)  In the most common version, the trajectories (movements) of atoms and molecules are determined by numerically solving Newton's equations of motion for a system of interacting particles, where forces between the particles and their potential energies are often calculated using interatomic potentials or molecular mechanics force fields." [wikipedia](https://en.wikipedia.org/wiki/Molecular_dynamics)

In this exercise we will perform a MD simulation of a single very simple molecule, namely methanol (CH3OH). The propagation of the atomic positions in time is usually treated classically and is given by Newton's Equation for an ensemble of particles:

$$ F(X) = - \nabla U(X) = M \dot{V}(t)$$

$$ V(t) = \dot{X}(t) $$

A molecular dynamics simulation therfore requires the definition of a potential function $U(X)$, or a description of the force terms $F(X)$ by which the particles in the simulation will interact. To correctly capture the molecular interactions, different methods are used for MD simulations (different methods of calculating the energy of atoms and then calculating the forces between them), depending on system. The most common are:

* Force fields in chemistry are empirical and consist of a summation of bonded forces associated with chemical bonds, bond angles, and bond dihedrals, and non-bonded forces associated with van der Waals forces and electrostatic charge. They are the most used interaction modelling technique for MD.

* Pair potentials between particles, in which the total potential energy can be calculated from the sum of energy contributions between pairs of atoms. An example of such a pair potential is the non-bonded Lennard–Jones potential.

* Semi-empirical potentials are based on quantum methods, but use empirical formula to determine the energy contributions of the orbitals, e.g. tight-binding potentials

*  Ab Initio Molecular Dynamics (AIMD) use quantum mechanical methods for the energy potential, which is more accurate and can describe bond-breaking etc. but which is also much more expensive (needs more resources of computational hardwares).

* QM/MM methods are hybrids between quantum mechanical (QM) and molecular or classical mechanics (MM), where only a small (and important) part is QM.

The goal of this exercise is to derive the correct potential energy by a neural network, which is fast in prediction and, if trained on a dataset from quantum mechanical (QM) calculations, also ideally as precise a AIMD/QM methods. Only with QM calculations effects such as bond breaking and reactions can be captured in a MD simulation. To work for arbitrary molecules and inter-molecular interactions the neural network potentials have to be convolutional (e.g. deep convolutional filter or graph networks) or atom-centered but which goes beyond the scope of this exercise. 

Question: Why do we want to use Neural networks for MD simulations?

1. Because Neural networks are always cool.
2. Neural networks are fast in prediction and in principle differentiable and could have QM accuracy if trained on QM data.
3. Force Fields do not work in MD simulation

In [ ]:
answer_md = 0 # (example), please pick your answer in this format
# YOUR CODE HERE
answer_md = 2
#raise NotImplementedError()

In [ ]:
# 1 Point
assert answer_md != 0

We start with loading and inspecting different geometries of methanol that has been sampled via distorting the molecule (the random distortiaon of the molecule gives many differnt molecules with diffetn shapes, in which the position of the atoms of the moleules is given in text file). A very common format are `.xyz` files. The format of a single xyz-file is:
1. 1st Line:              Number of atoms  (data format: integer)
2. 2nd Line:              comment         (data format: string)
3. 3rd Line and following: Elements and x- y- z-coordinates (data format: string and float, respectively)

In [ ]:
# importing the necessary python libraries
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
molecule_name = "methanol"

In [ ]:
# Defining a function which reads the xyz file and put the data in the value list
def load_csv(filepath, delimeter=" "):
    values = []
    # open file
    infile = open(filepath,"r")
    lines = infile.readlines()
    # read separate entries
    for line in lines:
        line_list = line.strip().split(delimeter)
        line_list = [x.strip() for x in line_list if x != '']
        values.append(line_list)
    # close file
    infile.close()
    return values

In [ ]:
# Here we show the first 30 elements (list elelemts, not the atoms) of the "values" list
# The 1st element is the number of the atoms in the first molecule, the 2nd one is a an empty line, 
# the 3rd element is a list of:'C' and its coordinates...
lines = load_csv(molecule_name+"_conformers.xyz")
lines[:30]

Now your task is to split separated lines into a nested list that runs over the molecules and looks like the example below. The coordinates shoud be given as floats and the comment line removed. The number of atoms tells you how many lines to add to the list. Note: your function should also work for arbitrary number of molecules and with different number of atoms in the same xyz-file.
```
[[['C', -0.371, 0.0077, -0.0086],
  ['O', 0.9167, -0.5048, -0.2972],
  ['H', -0.52, 0.0328, 1.0737],
  ['H', -0.465, 1.0143, -0.4234],
  ['H', -1.1223, -0.6427, -0.4626],
  ['H', 1.5617, 0.0926, 0.1181]],
 [['C', 0.0, 0.0, 0.0],
  ['O', 1.4629, 0.0, 0.0],
  ['H', -0.2012, 0.0, -1.0612],
  ['H', -0.3985, -0.9273, 0.1895],
  ['H', -0.4742, 0.6765, 0.6723],
  ['H', 1.6591, -0.7102, -0.5943]], ...]
```

In [ ]:
def lines_to_xyz(values):
    # Parse mols with flexible size
    # YOUR CODE HERE
    convert_list = []
    molecule = []

    for line in values:
        if not line:
            continue  # Skip empty lines
        if len(line) == 1:
            # New molecule
            if molecule:
                convert_list.append(molecule)
                molecule = []
        else:
            # Atom coordinates
            atom = [line[0]]
            atom.extend(float(coord) for coord in line[1:])
            molecule.append(atom)

    if molecule:
        convert_list.append(molecule)

    #raise NotImplementedError()
    return convert_list

In [ ]:
mols = lines_to_xyz(lines)

In [ ]:
# 2 Points
assert np.sum(np.abs(np.array(mols[1][2][1:]) - np.array([-0.2012, 0.0, -1.0612]))) < 0.001
assert mols[0][1][0] == 'O' and mols[0][3][0] == 'H'

In [ ]:
# 2 Points
assert lines_to_xyz([['1'],['my Comment'],['C','0.0','0.0','0.0'],
              ['2'],['my Comment'],['C','0.0','0.0','0.0'],['O','1.0','1.0','1.0']])[1][1][0] == 'O'
assert lines_to_xyz([['1'],['my Comment'],['C','0.0','0.0','0.0'],
              ['2'],['my Comment'],['C','0.0','0.0','0.0'],['O','1.0','1.0','1.0']])[1][1][1] - 1.0 < 1e-5

# 1. Learn Energies and Gradients of the molecule

## 1.1 Load Data

So that you can continue without solving previous exercise, load the numpy arrays below.

In [ ]:
# Already prepared. In case that you have not got the necessary data structure for the rest of the notebook
import numpy as np
geos = np.load(molecule_name+"_coordinates.npy") # in A
energies = np.load(molecule_name+"_energy.npy") # in eV
grads = np.load(molecule_name+"_gradients.npy")*27.21138624598853/0.52917721090380 # from H/B to eV/A

## 1.2 Train-Test-Split

In [ ]:
# Calling the libraries for splitting the dataset, and also using the evaluation metrics for the modelling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import tensorflow as tf

In [ ]:
# Scale energy (to normalize the dataset) and make test split
scaler = StandardScaler(with_std=True, with_mean=True, copy=True)

energy_scaled = scaler.fit_transform(energies[:, np.newaxis])
grads_scaled = grads / scaler.scale_

train_x, test_x, train_g, test_g, train_e, test_e = train_test_split(geos, grads_scaled, energy_scaled, test_size=0.2, shuffle=True, random_state=42)
print(train_x.shape)
print(train_g.shape)
print(train_e.shape)
# The coordinate value for each molecule contains 6 atoms and 3 xvz. The gradiant value for each moelcue...
# ...contains the gradiant of emenrgy for each of 6 atoms in x, y and z direction. However the energy is a global...
# ...value for each molecule and is not for atoms, and it doesnt have a direction.

Now in principle you could already set up a model with TensorFlow-Keras as shown below which simply takes all coordinates as input. You can test it with the data above, but please remove your code again for submission. 
Please have a look at the tensorflow api documentation: https://www.tensorflow.org/api_docs

Here we created a tensorflow model by sequentially setting up the layers of the model. The input tensor is passed from layer to layer within the model. The first dimension is always the batch dimension.
A fully connected neural network is given by `tf.keras.layers.Dense`. With the input-layer we define the input shape from which the hidden weights can be built (Note: you have to call the model once for the weights to actually being initalized). Since we want to use the neural network to model a smooth potential, we will not use "relu" but "selu". We use a regularizer for the kernel-weights.

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(6,3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300,use_bias=True, activation="selu", kernel_regularizer=tf.keras.regularizers.L1(1e-5)))
model.add(tf.keras.layers.Dense(300,use_bias=True, activation="selu", kernel_regularizer=tf.keras.regularizers.L1(1e-5)))
model.add(tf.keras.layers.Dense(1,use_bias=True,activation="linear"))
model.summary()

What could be the problem in this approach? Choose one of the anserwers:

1. The model does not have trainable weights.
2. The model can not deal with input tensor of rank > 1.
3. The model has input features that are not rotation and translation invariant but the model output is.

In [ ]:
answer_model_1 = 0 # please pick an answer as int
# YOUR CODE HERE
# Kontrolle: Ich schreibe mal 3 damit das assert nicht heult und weil 1 falsch ist und 2 ebenfalls falsch klingt
answer_model_1 = 3
#raise NotImplementedError()

In [ ]:
# 1 Point
assert answer_model_1 != 0

## 1.3 Features

With deep-learning APIs like TensorFlow, gradients and jacobians can be computed analytically. We make use of this to compute gradients or forces based on the Neural Network potential. This is further helpful since the gradients are vector properties and will depend on the orientation (not translation) of the molecule. However, to this end, we have to compute the feature descriptors within the model. 

We will do this within a layer named `InverseDistances`. The subclassed layer backbone is shown below. The backward pass is completely taken care of by tensorflow, as long as you use tensorflow functions that have a gradient defined: https://www.tensorflow.org/guide/autodiff

In [ ]:
class InverseDistances(tf.keras.layers.Layer):
    
    def __init__(self, pair_indices = None, **kwargs):
        super(InverseDistances, self).__init__(**kwargs)
        
        self.pair_indices = pair_indices

    def build(self, input_shape):
        
        if self.pair_indices is None:
            self.pair_indices = np.array([[i,j] for i in range(input_shape[1]) for j in range(i)], dtype=np.int64)
        else:
            self.pair_indices = np.array(self.pair_indices, dtype=np.int64)
            
        self.tf_pair_indices = self.add_weight('pair_indices',
                                               shape=self.pair_indices.shape,
                                               initializer=tf.keras.initializers.Zeros(),
                                               dtype='int64',
                                               trainable=False)

        super(InverseDistances, self).build(input_shape)
        
        self.set_weights([self.pair_indices])
        
    def call(self, inputs, **kwargs):
        
        # Translate the numpy version of the forward layer pass to tensorflow functions
        # Find the appropriate tensorflow functions. Everything must be vectorized.
        # YOUR CODE HERE
        cordbatch = inputs
        indexbatch = self.tf_pair_indices
        # indexbatch = tf.expand_dims(self.tf_pair_indices,axis=0)
        v1 = tf.gather(cordbatch, indexbatch[:,0], axis=1, batch_dims=0)
        v2 = tf.gather(cordbatch, indexbatch[:,1], axis=1, batch_dims=0)
        vec = v2 - v1  
        norm_vec = tf.sqrt(tf.reduce_sum(tf.square(vec), axis=-1))
        invd_out = tf.math.reciprocal(norm_vec)
        #raise NotImplementedError()
        return invd_out
    
    def _call_numpy_version_not_use(self, inputs):
        # The same as call with numpy
        cordbatch = inputs
        indexbatch = self.pair_indices # You may have to add a dimension to indices for tf
        v1 = np.take(cordbatch,indexbatch[:,0],axis=1) # You have to find a solution with tf.gather here
        v2 = np.take(cordbatch,indexbatch[:,1],axis=1) # For tf check out the batch_dims and axis parameter for tf.gather
        vec = v2-v1
        norm_vec = np.sqrt(np.sum(vec*vec,axis=-1))
        invd_out = 1/norm_vec
        return invd_out

In [ ]:
# Test layer
test_layer = InverseDistances(dtype='float64')
test_layer.build(geos[:10,:,:].shape)
result_test_layer = test_layer(geos[:10,:,:]).numpy()
result_test_numpy = test_layer._call_numpy_version_not_use(geos[:10,:,:])
result_test_numpy.shape

In [ ]:
# 3 Points
assert np.sum(np.abs(result_test_layer-result_test_numpy))< 1e-6

In principle we can make now a model like shown below. However, it turns out that training with the prediction of the gradients leads to overall better results. Note the non-trainable weights for the indices that come from our custom layer.

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(6,3)))
model.add(InverseDistances(name="InverseDistance"))
model.add(tf.keras.layers.Dense(300,use_bias=True, activation="selu", kernel_regularizer=tf.keras.regularizers.L1(1e-5)))
model.add(tf.keras.layers.Dense(300,use_bias=True, activation="selu", kernel_regularizer=tf.keras.regularizers.L1(1e-5)))
model.add(tf.keras.layers.Dense(1,use_bias=True,activation="linear"))
model.summary()

Why could it be more advantageous to train with energy AND forces as model output?

1. The forces are not connected to the energies and therefore needs to be in the training set.
2. Adding the forces makes training much faster
3. The gradients determine the slope of the potential energy surface and act as an additional form of regularization.

In [ ]:
answer_gradient = 0 # select the number of the correct answer
# YOUR CODE HERE
# Korrektur:
answer_gradient = 3
#raise NotImplementedError()

In [ ]:
# 1 Point
assert answer_gradient != 0

## 1.4 Energy + Gradients

We can improve the gradient prediction if we train on energies and gradients simultaneously. To do this the analytical gradient prediction has to be integrated into the model. A subclassing of a tf.keras.model allows for implementing a more general model definition than the simply sequential model constructor.

In [ ]:
class EnergyGradientModel(tf.keras.Model):

    def __init__(self, **kwargs):

        super(EnergyGradientModel, self).__init__(**kwargs)

        self.feat_layer = InverseDistances()
        
        self.dense_layers = [ tf.keras.layers.Dense(300,use_bias=True, activation="selu", kernel_regularizer=tf.keras.regularizers.L1(1e-5)),
                          tf.keras.layers.Dense(300,use_bias=True, activation="selu", kernel_regularizer=tf.keras.regularizers.L1(1e-5))
                          ]

        self.energy_layer = tf.keras.layers.Dense(1,use_bias=True,activation="linear")
        
    def call(self, inputs, training=False, **kwargs):
        
        x = inputs
        with tf.GradientTape() as tape:
            tape.watch(x)
            hidden = self.feat_layer(x)
            for d in self.dense_layers:
                hidden = d(hidden, training=training)
            temp_e = self.energy_layer(hidden)
        temp_g = tape.batch_jacobian(temp_e,x)
        temp_g = temp_g[:,0,:,:]
        return [temp_e, temp_g]

In [ ]:
model = EnergyGradientModel()
model.build(train_x.shape)

In [ ]:
# Compile and Training of the model
def compile_train_model(model,x,y,
                        validation_data=None,
                        epochs=1000,
                        lr=0.5e-3,
                        validation_freq=10,
                        batch_size=128,
                        verbose=2,
                        loss='mean_squared_error',
                        metrics=['mean_absolute_error'],
                        loss_weights=None):

    # Compile model with optimizer and learning rate
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=metrics,
                  loss_weights=loss_weights)

    hist = model.fit(x,y,epochs=epochs,
                     batch_size=batch_size,
                     validation_freq=validation_freq,
                     validation_data=validation_data,
                     verbose=2)
    
    return hist

Please submit your solution with `do_training = False`.

In [ ]:
do_training = False
# # YOUR CODE HERE
# # Kontrolle: Auskommentieren vor abgabe
# do_training = True
# #raise NotImplementedError()

In [ ]:
if do_training:
    hist = compile_train_model(model,train_x,
                               [train_e, train_g],
                               validation_data=(test_x,[test_e, test_g]),
                               loss=["mean_squared_error","mean_squared_error"],
                               metrics=[['mean_absolute_error'],['mean_absolute_error']],
                               loss_weights=[1,5]
                              )

In [ ]:
# Some plotting functions.
def plot_history(hist, validation_freq=10,scale=1):    
    plt.figure()
    for key, loss in hist.history.items():
        np_loss=np.array(loss)
        if "val" in key:
            plt.plot(np.arange(np_loss.shape[0])*validation_freq+validation_freq,np_loss, label=key)
        else:
            plt.plot(np.arange(np_loss.shape[0]), np_loss, label=key)
        
    plt.xlabel('Epochs')
    plt.ylabel('Loss ')
    plt.title('Loss vs. epochse')
    plt.legend(loc='upper right', fontsize='x-small')
    plt.show()

def plot_prediction(pred,true):
    mae_valid = np.mean(np.abs(pred - true))
    r2_data = r2_score(true, pred)
    print("MAE", mae_valid)
    print("r2_score", r2_data)
    plt.figure()
    plt.scatter(pred.flatten(), true.flatten(), alpha=0.3, label="MAE: {0:0.4f} \nr2 {1:0.4f}".format(mae_valid, r2_data))
    plt.plot(np.linspace(np.amin(true), np.amax(true), 100),
             np.linspace(np.amin(true), np.amax(true), 100), color='red')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.legend(loc='upper left', fontsize='x-large')
    plt.show()

In [ ]:
if do_training:
    pred_test = scaler.inverse_transform(model.predict(test_x)[0])
    true_test = scaler.inverse_transform(test_e)
    pred_test_grad = model.predict(test_x)[1]*scaler.scale_
    true_test_grad = test_g*scaler.scale_

    plot_history(hist, validation_freq=10, scale=scaler.scale_)
    plot_prediction(pred_test,true_test)
    plot_prediction(np.reshape(pred_test_grad,(-1,18)),np.reshape(true_test_grad,(-1,18)))

Please report on your achieved validation r2_score. You can change the hyperparemeters and play with the model definition. Eg. add a few layers or add a BatchNormalization layer after the Feature layer etc.

In [ ]:
r2_score_energy = 0
r2_score_gradient = 0
# YOUR CODE HERE

r2_score_energy = 0.9615625688182862
r2_score_gradient = 0.9959105940536966

#raise NotImplementedError()

In [ ]:
# 3 Points
assert r2_score_energy > 0.90
assert r2_score_gradient > 0.90

# MD Simulation

For time integration the so-called ["Verlet"](https://en.wikipedia.org/wiki/Verlet_integration) integration is used to integrate Newton's equations of motion.

The standard scheme of this algorithm is:

* Calculate ${\displaystyle {\vec {v}}\left(t+{\tfrac {1}{2}}\,\Delta t\right)={\vec {v}}(t)+{\tfrac {1}{2}}\,{\vec {a}}(t)\,\Delta t}$.

* Calculate ${\displaystyle {\vec {x}}(t+\Delta t)={\vec {x}}(t)+{\vec {v}}\left(t+{\tfrac {1}{2}}\,\Delta t\right)\,\Delta t}$.

* Derive ${\displaystyle {\vec {a}}(t+\Delta t)}$ from the interaction potential using ${\displaystyle {\vec {x}}(t+\Delta t)}$.

* Calculate ${\displaystyle {\vec {v}}(t+\Delta t)={\vec {v}}\left(t+{\tfrac {1}{2}}\,\Delta t\right)+{\tfrac {1}{2}}\,{\vec {a}}(t+\Delta t)\Delta t}$.

Here is an implementation in python with different time-integration.

In [ ]:
class PotentialNN:
    
    unit_Bohr_A = 0.52917721090380
    unit_Hatree_eV = 27.21138624598853
    
    def __init__(self, model, scaler):
        self.model = model
        self.scaler = scaler
    
    def __call__(self,x):
        # x in Bohr to eV
        x_A = x*self.unit_Bohr_A
        # call model
        eng, grad = self.model.predict(x_A)
        # inverse scaling
        eng = self.scaler.inverse_transform(eng)
        grad = grad*self.scaler.scale_
        # unit conversion to Hatree and Hatree/Bohr
        eng_B = eng/self.unit_Hatree_eV
        grad_BH = grad/self.unit_Hatree_eV*self.unit_Bohr_A
        return eng_B, grad_BH

In [ ]:
class BatchEnsemble:
    
    unit_Bohr_A = 0.52917721090380
    unit_Bohr_m = 5.2917721090380e-11
    unit_me_kg = 9.109383701528e-31
    unit_Hatree_eV = 27.21138624598853
    unit_Hatree_J = 4.359744722207185e-18
    unit_atu_s = 2.418884326585747e-17
    unit_1u_me = 1.6605390666050e-27/9.109383701528e-31
    
    const_kB = 8.617333262e-5/27.21138624598853
    sign_force = -1.0
    
    def __init__(self, coord, mass, potential, velo = None):
        """Initial settings. All properties are in atomic units.
        
        Args:
            coord (np.array): Initial poisition of shape (batch, N, 3) in [Bohr]
            mass (np.array): Mass of particles (bath, N, 1) in [me]
            potential (callable): Potential model inputs coord and returns energies and gradients
                                  of shape (batch, 1) and (batch, N, 3) in [Hatree] and [Hatree/Bohr]
            velo (np.array): Initial velocities of shape (batch, N, 3)
        """
        
        # System properties
        self.mass = mass
        self.potential = potential
        self.number_particles = self.mass.shape[1]
        
        # Trajectory properties
        self.traj_x = [] # position in Bohr
        self.traj_v = [] # velicity in Bohr/atu
        self.traj_a = [] # acceleraton in Bohr/atu^2
        self.traj_t = [] # time in atu
        self.traj_p = [] # momentum in Bohr*me/atu
        self.traj_F = [] # force in Hatree/Bohr
        self.traj_E = [] # potential energy in Hatree
        self.traj_E_kin = [] # kinetic energy in Hatree
        self.traj_T = [] # Temperature in K
        
        #######################################################################
        
        # Set initial values i.e. traj[0]
        initial_x = coord
        if velo is None:
            initial_v = np.zeros_like(coord)
        else:
            initial_v = velo
        initial_p = initial_v*self.mass
        initial_eng, initial_grad = self.potential(initial_x)
        initial_force = self.sign_force*initial_grad
        initial_eng_kin = np.sum(np.sum(0.5*self.mass*initial_v*initial_v,axis=-1),axis=-1,keepdims=True)
        initial_temp = 2/3*initial_eng_kin/self.number_particles/self.const_kB
        initial_a = initial_force/self.mass
        
        # Append 0 time step
        self.traj_x.append(initial_x)
        self.traj_v.append(initial_v)
        self.traj_E.append(initial_eng)
        self.traj_F.append(initial_force)
        self.traj_a.append(initial_a)
        self.traj_t.append(0.0)
        self.traj_E_kin.append(initial_eng_kin)
        self.traj_T.append(initial_temp)
        self.traj_p.append(initial_p)
    
    def initialize_velocity(self, T):
        """Overwrite initial velcovity by Boltzmann distribution.
        
        Args:
            T (float): Temperature in K
        """
        initial_velo = np.random.standard_normal(self.traj_x[0].shape)
        initial_velo = initial_velo*np.sqrt(self.const_kB*T/self.mass)
        initial_eng_kin = np.sum(np.sum(0.5*self.mass*initial_velo*initial_velo,axis=-1),axis=-1,keepdims=True)
        initial_p = initial_velo*self.mass
        initial_T = 2/3*initial_eng_kin/self.number_particles/self.const_kB
        
        self.traj_v[0] = initial_velo
        self.traj_E_kin[0] = initial_eng_kin
        self.traj_T[0] = initial_T
        self.traj_p[0] = initial_p

In [ ]:
class TimeIntegration:
     
    unit_atu_s = 2.418884326585747e-17
    const_kB = 8.617333262e-5/27.21138624598853
    
    def __init__(self,ensemble):
        self.ensemble = ensemble
    
    def propagate_timestep(self):
        pass

    def propagate(self, time_length, step_size):
        """Propagate ensemble.
        
        Args:
            time_length: Time of the simulation in [fs]
            step_size: Time step in [fs]        
        """
        #Repeat time step
        num_steps = int(time_length/step_size)
        
        # Map to atu
        time_length = time_length*1e-15/self.unit_atu_s
        step_size = step_size*1e-15/self.unit_atu_s
        print("Run MD for:", time_length, "a.t.u with steps:" ,step_size, "a.t.u")
        
          
        propagation_function = self.propagate_timestep
        
        # Run MD
        for i in range(num_steps):
            propagation_function(step_size)  
            if i%500==0:
                print("Steps done:",i)

In [ ]:
class VerletIntegration(TimeIntegration):

    def propagate_timestep(self, delta_t):
        # time in atu
        
        t = self.ensemble.traj_t[-1] 
        x_t = self.ensemble.traj_x[-1]
        a_t = self.ensemble.traj_a[-1]
        v_t = self.ensemble.traj_v[-1]
        m = self.ensemble.mass
        N = self.ensemble.number_particles
        kB = self.const_kB
        sig_F = self.ensemble.sign_force
        
        v_t_dt_2 = v_t + 0.5*a_t*delta_t
        x_t_dt = x_t + v_t_dt_2*delta_t 
        
        e_t_dt, g_t_dt = self.ensemble.potential(x_t_dt)
        a_t_dt = sig_F*g_t_dt/m
        
        v_t_dt = v_t_dt_2 + 0.5*a_t_dt*delta_t
        e_kin_t_dt = np.sum(np.sum(0.5*m*v_t_dt*v_t_dt,axis=-1),axis=-1,keepdims=True)
        p_t_dt = v_t_dt*m
        T_dt = 2/3*e_kin_t_dt/N/kB

        # Add time-step
        self.ensemble.traj_x.append(x_t_dt)
        self.ensemble.traj_v.append(v_t_dt)
        self.ensemble.traj_a.append(a_t_dt)
        self.ensemble.traj_t.append(t + delta_t)
        self.ensemble.traj_F.append(sig_F*g_t_dt)
        self.ensemble.traj_E.append(e_t_dt)
        self.ensemble.traj_E_kin.append(e_kin_t_dt)
        self.ensemble.traj_T.append(T_dt)
        self.ensemble.traj_p.append(p_t_dt)

In [ ]:
class BerendsenThermostat(TimeIntegration): 
    
    def __init__(self,ensemble,f_cool = 0,T0 = 300):
        """Make Time Integration with Berendsen Thermostat for Verlet.
        
        Args:
            ensemble: BatchEnsemble class
            f_cool: Colling coupling as frequency in [fs]
            T0: Temperature of bath [K]
        """
        self.ensemble=ensemble
        self.f_cool = f_cool/1e-15*self.unit_atu_s
        self.T0 = T0

    def propagate_timestep(self, delta_t):
        # time in atu
        
        t = self.ensemble.traj_t[-1] 
        x_t = self.ensemble.traj_x[-1]
        a_t = self.ensemble.traj_a[-1]
        v_t = self.ensemble.traj_v[-1]
        m = self.ensemble.mass
        N = self.ensemble.number_particles
        kB = self.const_kB
        sig_F = self.ensemble.sign_force
        f_cool = self.f_cool
        
        v_t_dt_2 = v_t + 0.5*a_t*delta_t
        
        e_kin_t_dt_2 = np.sum(np.sum(0.5*m*v_t_dt_2*v_t_dt_2,axis=-1),axis=-1,keepdims=True)
        T_dt_2 = 2/3*e_kin_t_dt_2/N/kB
        lamd = np.sqrt(1+f_cool*delta_t*(self.T0/T_dt_2-1))
        
        x_t_dt = x_t + v_t_dt_2*delta_t 
        
        e_t_dt, g_t_dt = self.ensemble.potential(x_t_dt)
        a_t_dt = sig_F*g_t_dt/m
        
        v_t_dt = v_t_dt_2 + 0.5*a_t_dt*delta_t
        v_t_dt = np.expand_dims(lamd,axis=-1)*v_t_dt
        e_kin_t_dt = np.sum(np.sum(0.5*m*v_t_dt*v_t_dt,axis=-1),axis=-1,keepdims=True)
        p_t_dt = v_t_dt*m
        T_dt = 2/3*e_kin_t_dt/N/kB

        # Add time-step
        self.ensemble.traj_x.append(x_t_dt)
        self.ensemble.traj_v.append(v_t_dt)
        self.ensemble.traj_a.append(a_t_dt)
        self.ensemble.traj_t.append(t + delta_t)
        self.ensemble.traj_F.append(sig_F*g_t_dt)
        self.ensemble.traj_E.append(e_t_dt)
        self.ensemble.traj_E_kin.append(e_kin_t_dt)
        self.ensemble.traj_T.append(T_dt)
        self.ensemble.traj_p.append(p_t_dt)

Now we can run a very simple MD simulation for a batch of molecules. We take the energy minimum and initialize a batch of molecules with temperature T. We also have to provide mass and the neural network potential in matching units. Please submit you notebook with `do_poropgate = False` and `show_trajectory = False`.

In [ ]:
do_poropgate = False
show_trajectory = False
## YOUR CODE HERE
## Kontrolle: Auskommentieren vor abgabe
#do_poropgate = True
#show_trajectory = True
##raise NotImplementedError()

In [ ]:
# Make a batch molecules.
lowest = np.argsort(energies)[0]
print("Energy Minimum",energies[lowest])
x_md = geos[lowest]
x_md = np.repeat(np.expand_dims(x_md,axis=0),10,axis=0)
x_md = x_md/PotentialNN.unit_Bohr_A
mass = np.array([[[12.0],[15.99491],[1.007825],[1.007825],[1.007825],[1.007825]]])*BatchEnsemble.unit_1u_me

# Setup Ensemble and 
potential = PotentialNN(model,scaler)
ensemble_md = BatchEnsemble(x_md, mass, potential)
ensemble_md.initialize_velocity(1000.0)
trajectory_md = VerletIntegration(ensemble_md)

In [ ]:
if do_poropgate:
    trajectory_md.propagate(1000,1)
    plt.figure(figsize=(15,5))
    plt.subplot(121)
    for i in range(10):
        plt.plot(np.array(ensemble_md.traj_t),np.array(ensemble_md.traj_E)[:,i,0], label=str(i))
    plt.legend(loc="right")
    plt.xlabel("Time [a.t.u]")
    plt.ylabel("Potential Energy [Hatree]")
    plt.subplot(122)
    for i in range(10):
        plt.plot(np.array(ensemble_md.traj_t),np.array(ensemble_md.traj_T)[:,i,0], label=str(i))
    plt.legend(loc="right")
    plt.xlabel("Time [a.t.u]")
    plt.ylabel("Temperature [K]")
    plt.ylim([0,2000])
    plt.show()

We can look at a molecule from the batch with the `ase` viewer. You should see a new pannel where you can hit play and choose the number of frames per second.

In [ ]:
if show_trajectory:
    import ase
    from ase.visualize import view
    traj_pos_md = np.array(ensemble_md.traj_x)[:,0,:,:]*0.52917721090380
    traj_pos_md_ase = [ase.Atoms(['C', 'O', 'H', 'H', 'H', 'H'],positions=x) for x in traj_pos_md]
    view(traj_pos_md_ase)

You notice a large spread in the initial temperatures. Have a look at the method `initialize_velocity` in the definition of the `BatchEnsemble` class. What could be a better way to initialized the velocities to achieve a specific kinetic energy or temperature. 

1. Set all $\forall v \in \mathbb{R}^3: v_{x,y,z} = v_0 $ constant.
2. Choose a $|v|$ from Maxwell-Boltzmann distribution and a random direction for each atom, remove the COM velocity and rescale according to desired temperature.
3. Set all $v = 0$ and adjust potential scale.

In [ ]:
answer_md_v = 0 # choose the correct answer
# YOUR CODE HERE
# Kontrolle: 
answer_md_v = 2
#raise NotImplementedError()

In [ ]:
# 1 Points
assert answer_md_v != 0

Now we run the MD Simulation with a thermostat. That means we can cool the system during the simulation.

In [ ]:
potential = PotentialNN(model,scaler)
ensemble_md = BatchEnsemble(x_md, mass, potential)
ensemble_md.initialize_velocity(1000.0)
trajectory_md = BerendsenThermostat(ensemble_md,f_cool = 0.01, T0 = 5)

In [ ]:
if do_poropgate:
    trajectory_md.propagate(2000,1)
    plt.figure(figsize=(15,5))
    plt.subplot(121)
    for i in range(10):
        plt.plot(np.array(ensemble_md.traj_t),np.array(ensemble_md.traj_E)[:,i,0], label=str(i))
    plt.legend(loc="right")
    plt.xlabel("Time [a.t.u]")
    plt.ylabel("Potential Energy [Hatree]")
    plt.subplot(122)
    for i in range(10):
        plt.plot(np.array(ensemble_md.traj_t),np.array(ensemble_md.traj_T)[:,i,0], label=str(i))
    plt.legend(loc="right")
    plt.xlabel("Time [a.t.u]")
    plt.ylabel("Temperature [K]")
    plt.ylim([0,2000])
    plt.show()

In [ ]:
if show_trajectory:
    import ase
    from ase.visualize import view
    traj_pos_md = np.array(ensemble_md.traj_x)[:,0,:,:]*0.52917721090380
    traj_pos_md_ase = [ase.Atoms(['C', 'O', 'H', 'H', 'H', 'H'],positions=x) for x in traj_pos_md]
    view(traj_pos_md_ase)

Compare the time evolution of the simulation with Verlet integration to the one with Berendsen thermostat, which one shows the decrease in the temperature and energy of the system? Answer with a string "A" for the Verlet and "B" for the Berendsen. Hint: you could also check the visualization and observe the molecule's vibration over time.

In [ ]:
answer_observation = ""
# YOUR CODE HERE
answer_observation = "B"
# raise NotImplementedError()

In [ ]:
# 1 Point
assert isinstance(answer_observation, str)

In [ ]:
# 3 Points
assert do_training == False
assert do_poropgate == False
assert show_trajectory == False